In [ ]:
import numpy as np
import cv2
import random
import heapq
import tkinter as tk
from PIL import Image, ImageTk, ImageDraw, ImageFont
from IPython.display import display
import random
import math
import matplotlib.pyplot as plt
import pygame
import time
from collections import OrderedDict
import sys







# def generate_goals(num_goals, height, width, existing_goals=None):
#     goals = existing_goals if existing_goals is not None else set()
    
#     while len(goals) < num_goals:
#         row = random.randrange(height)
#         col = random.randrange(width)
#         if (row, col) not in goals:
#             goals.add((row, col))
    
#     return goals

def generate_goals(num_goals, height, width, existing_goals=None):
    goals = existing_goals if existing_goals is not None else []
    
    while len(goals) < num_goals:
        row = random.randrange(height)
        col = random.randrange(width)
        if (row, col) not in goals:
            goals.append((row, col))
    
    return goals



class StateSpace:
    def __init__(self, goals):
        """Create a new state space with given dimensions."""
        self.height = 15
        self.width = 15
        
        self.num_hospitals = 8
        self.num_houses = 8
        self.num_buildings = 2
        self.num_pisa = 2
        self.num_jeep = 8
        self.num_bike = 8
        
        self.houses = set()
        self.hospitals = set()
        self.buildings = set()
        self.pisa = set()
        self.jeep = set()
        self.bike = set()
        self.candidates = set()
        
        self.goals = goals
        self.start = {(0,0)}

        # Add houses
        while len(self.houses) < self.num_houses:
            row = random.randrange(self.height)
            col = random.randrange(self.width)
            if (row, col) not in self.houses and (row, col) not in self.start and (row,col) not in self.goals:
                self.houses.add((row, col))

        # Add hospitals
        while len(self.hospitals) < self.num_hospitals:
            row = random.randrange(self.height)
            col = random.randrange(self.width)
            if (row, col) not in self.hospitals and (row, col) not in self.houses and (row, col) not in self.start and (row, col) not in self.goals:
                self.hospitals.add((row, col))

        # Add buildings
        while len(self.buildings) < self.num_buildings:
            row = random.randrange(self.height)
            col = random.randrange(self.width)
            if (row, col) not in self.hospitals and (row, col) not in self.houses and (row, col) not in self.buildings and (row, col) not in self.start and (row, col) not in self.goals:
                self.buildings.add((row, col))

        # Add Pisa
        while len(self.pisa) < self.num_pisa:
            row = random.randrange(self.height)
            col = random.randrange(self.width)
            if (row, col) not in self.hospitals and (row, col) not in self.houses and (row, col) not in self.buildings and (row, col) not in self.pisa and (row, col) not in self.start and (row, col) not in self.goals:
                self.pisa.add((row, col))

        # Add Jeep
        while len(self.jeep) < self.num_jeep:
            row = random.randrange(self.height)
            col = random.randrange(self.width)
            if (row, col) not in self.hospitals and (row, col) not in self.houses and (row, col) not in self.buildings and (row, col) not in self.pisa and (row, col) not in self.jeep and (row, col) not in self.start and (row, col) not in self.goals:
                self.jeep.add((row, col))

        # Add Bike
        while len(self.bike) < self.num_bike:
            row = random.randrange(self.height)
            col = random.randrange(self.width)
            if (row, col) not in self.hospitals and (row, col) not in self.houses and (row, col) not in self.buildings and (row, col) not in self.pisa and (row, col) not in self.jeep and (row, col) not in self.bike and (row, col) not in self.start and (row, col) not in self.goals:
                self.bike.add((row, col))

        # Consider all possible cells
        self.candidates = {(row, col) for row in range(self.height) for col in range(self.width)}

        # Remove all houses, hospitals, buildings, Pisa, Jeep, and Bike from candidates
        self.candidates -= self.houses
        self.candidates -= self.hospitals
        self.candidates -= self.buildings
        self.candidates -= self.pisa
        self.candidates -= self.jeep
        self.candidates -= self.bike
        self.candidates -= self.goals
        self.candidates -= self.start


class Node:
    def __init__(self, state, parent, action, priority=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.priority = priority
        #self.cost = cost

    def __lt__(self, other):
        return self.priority < other.priority

class PriorityQueue:
    def __init__(self):
        self.elements = []

    def empty(self):
        return len(self.elements) == 0

    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))

    def get(self):
        return heapq.heappop(self.elements)[1]

class Maze:
    def __init__(self, goals, start, goalPoint):
        self.goal_list = goals
        self.goals_set = set(goals)
        ss = StateSpace(self.goals_set)
        self.height = ss.height
        self.width = ss.width
        self.walls = []
        self.start = start
        self.goalPoint = goalPoint
        self.goals = ss.goals
        self.hospitals = ss.hospitals
        self.houses = ss.houses  # Assign houses from StateSpace object
        self.buildings = ss.buildings
        self.pisa = ss.pisa
        self.jeep = ss.jeep
        self.bike = ss.bike

  


        for i in range(self.height):
            row = []
            for j in range(self.width):
#                 if (i, j) in self.goals and (i, j) != self.goalPoint:
#                     row.append(True)  # True if goal
                if (i, j) in self.hospitals or (i, j) in self.houses or (i, j) in self.buildings or (i, j) in self.pisa or (i, j) in self.jeep or (i, j) in self.bike:
                    row.append(True)
                else:
                    row.append(False)  # False if available space
            self.walls.append(row)

        self.solution = None
        self.exploredList = []


    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        
        # Define colors
        WHITE = (255, 255, 255)
        BLACK = (0, 0, 0)
        RED = (255, 0, 0)
        
        # Function to draw grid elements
        def draw_grid():
            screen.fill(WHITE)  # Fill the screen with white

            # Draw grid lines
            for x in range(0, width * cell_size, cell_size):
                pygame.draw.line(screen, BLACK, (x, 0), (x, height * cell_size))
            for y in range(0, height * cell_size, cell_size):
                pygame.draw.line(screen, BLACK, (0, y), (width * cell_size, y))

            # Draw other elements (start, goal, walls, etc.)
            for i, row in enumerate(self.walls):
                for j, col in enumerate(row):
                    if (i, j) == self.start:
                        image_path = "assets/images/rider.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.goal_list:
                        image_path = f"assets/images/{self.goal_list.index((i, j)) + 1}.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.hospitals:
                        image_path = "assets/images/Hospital.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.houses:
                        image_path = "assets/images/House.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.buildings:
                        image_path = "assets/images/building.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.pisa:
                        image_path = "assets/images/pisa.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.jeep:
                        image_path = "assets/images/jeep.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif (i, j) in self.jeep:
                        image_path = "assets/images/bike.png"
                        draw_image(image_path, j * cell_size, i * cell_size)
                    elif solution is not None and (i, j) in solution:
                        pygame.draw.rect(screen, BLACK, (j * cell_size, i * cell_size, cell_size, cell_size))
                        pygame.draw.rect(screen, WHITE, (j * cell_size, i * cell_size, cell_size, cell_size), 1)  # Add white border
                    elif solution is not None and (i, j) in self.explored and (i, j) not in solution:
                        pygame.draw.rect(screen, RED, (j * cell_size, i * cell_size, cell_size, cell_size))
                        pygame.draw.rect(screen, WHITE, (j * cell_size, i * cell_size, cell_size, cell_size), 1)  # Add white border

                        
                        
        # Function to draw an image
        def draw_image(image_path, x, y):
            image = pygame.image.load(image_path)
            image = pygame.transform.scale(image, (cell_size, cell_size))
            screen.blit(image, (x, y))

        # Initialize Pygame
        pygame.init()

        # Set up the display
        cell_size = 40
        width, height = 15, 15  # Define grid size
        screen = pygame.display.set_mode((width * cell_size, height * cell_size), pygame.RESIZABLE)
        pygame.display.set_caption("A* Pathfinding")

        # Draw initial grid
        draw_grid()
        pygame.display.flip()
        time.sleep(1)  # Add a delay for visualization

#         # Animation loop for exploring the path
#         if solution is not None:
#             for step in solution:
#                 draw_grid()
#                 # Draw rider image at the current position
#                 draw_image("assets/images/rider.png", step[1] * cell_size, step[0] * cell_size)
#                 pygame.display.flip()
#                 time.sleep(0.5)  # Add a delay for visualization

#         if step not in self.explored:
#             draw_grid()
#             # Draw rider image at the current position with red border
#             draw_image("assets/images/rider.png", step[1] * cell_size, step[0] * cell_size)
#             pygame.draw.rect(screen, RED, (step[1] * cell_size, step[0] * cell_size, cell_size, cell_size), 1)  # Add red border
#             pygame.display.flip()
#             time.sleep(0.5)  # Add a delay for visualization

    # Animation loop for exploring the path using self.exploredList
        if self.exploredList:
            # Remove duplicates from self.exploredList while preserving order
            self.exploredList = list(OrderedDict.fromkeys(self.exploredList))
            for step in self.exploredList:
                draw_grid()
                # Draw rider image at the current position
                draw_image("assets/images/rider.png", step[1] * cell_size, step[0] * cell_size)
                pygame.display.flip()
                time.sleep(0.5)  # Add a delay for visualization

                # Check if the step is not in self.explored and draw a red border
                if step not in self.explored:
                    pygame.draw.rect(screen, RED, (step[1] * cell_size, step[0] * cell_size, cell_size, cell_size), 1)
                    pygame.display.flip()
                    time.sleep(0.5)  # Add a delay for visualization



        # Keep the window open until closed by the user
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

        # Clean up resources
        pygame.quit() 
        
#     def print_sim(self):
#         solution = self.solution[1] if self.solution is not None else None
#         cell_size = 60
#         #goal_list = list(goals_set)  # Convert set to list for manipulation

#         img = Image.new("RGB", (self.width * cell_size, self.height * cell_size), "white")

#         for i, row in enumerate(self.walls):
#             for j, col in enumerate(row):
#                 if (i, j) == self.start:
#                     image_path = "assets/images/rider.png"
#                     self.add_image(img, image_path, cell_size, i, j)
# #                 elif (i, j) in self.goals:
# #                     image_path = "assets/images/delivery.png"
# #                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) == self.goal_list[0]:
#                     image_path = "assets/images/1.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) == self.goal_list[1]:
#                     image_path = "assets/images/2.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) == self.goal_list[2]:
#                     image_path = "assets/images/3.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) == self.goal_list[3]:
#                     image_path = "assets/images/4.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) == self.goal_list[4]:
#                     image_path = "assets/images/5.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif solution is not None and (i, j) in solution:
#                     # Print solution path
#                     self.add_label(img, "", "white", cell_size, i, j)
#                 elif solution is not None and (i, j) in self.explored and (i, j) not in solution:
#                     self.add_label(img, "", "red", cell_size, i, j)
#                 elif (i, j) in self.hospitals:
#                     # Print hospital image
#                     image_path = "assets/images/Hospital.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) in self.houses:
#                     # Print house image
#                     image_path = "assets/images/House.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) in self.buildings:
#                     image_path = "assets/images/building.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) in self.pisa:
#                     image_path = "assets/images/pisa.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) in self.jeep:
#                     image_path = "assets/images/jeep.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 elif (i, j) in self.jeep:
#                     image_path = "assets/images/bike.png"
#                     self.add_image(img, image_path, cell_size, i, j)
#                 else:
#                     # Print empty space
#                     self.add_label(img, "", "black", cell_size, i, j)

#         display(img)


    
    def add_label(self, img, text, color, size, row, col):
        draw = ImageDraw.Draw(img)
        x0 = col * size + 1  # Add 1 pixel padding to the left
        y0 = row * size + 1  # Add 1 pixel padding to the top
        x1 = x0 + size - 2   # Subtract 2 pixels for padding (1 pixel from left and 1 from right)
        y1 = y0 + size - 2   # Subtract 2 pixels for padding (1 pixel from top and 1 from bottom)
        draw.rectangle([x0, y0, x1, y1], fill=color, outline="white")  # Draw a rectangle with white border and specified color fill
        draw.text((x0 + size // 2, y0 + size // 2), text, fill="black")

    def add_image(self, img, image_path, size, row, col):
        padding = 1  # Padding around the image
        image = Image.open(image_path)
        image = image.resize((size - 2 * padding, size - 2 * padding), Image.LANCZOS)  # Subtract padding from width and height
        img.paste(image, (col * size + padding, row * size + padding))  # Add padding to the position

        

#     def neighbors(self, state):
#         row, col = state
#         candidates = [
#             ("up", (row - 1, col)),
#             ("down", (row + 1, col)),
#             ("left", (row, col - 1)),
#             ("right", (row, col + 1))
#         ]
#         result = []
#         for action, (r, c) in candidates:
#             if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
#                 result.append((action, (r, c)))
#         return result

    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up_left", (row - 1, col - 1)),
            ("up", (row - 1, col)),
            ("up_right", (row - 1, col + 1)),
            ("down_left", (row + 1, col - 1)),
            ("down", (row + 1, col)),
            ("down_right", (row + 1, col + 1)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]
        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result

    def euclidean_distance(self, state1, state2):
        x1, y1 = state1
        x2, y2 = state2
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    def greedy_bfs(self):
        self.num_explored = -1
        start = Node(state=self.start, parent=None, action=None)
        frontier = PriorityQueue()
#         cost = random.randint(1, 20)
        frontier.put(start, 0)
    
        # for simulation
       # self.exploredList = []
        self.explored = set()
        max_frontier_size = 0
        space_complexity = 0

        while True:
            if frontier.empty():
                raise Exception("no solution")

            node = frontier.get()
            self.num_explored += 1

            if node.state == self.goalPoint:
                self.exploredList.append(node.state)
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                # print("Space complexity (Maximum frontier size):", space_complexity)
                return

            self.explored.add(node.state)
            self.exploredList.append(node.state)

            for action, state in self.neighbors(node.state):
                if state not in self.explored:
                    
#                     cost = node.cost + random.randint(1, 20) 
                    #cost = node.cost + 1
                    priority = self.euclidean_distance(state, self.goalPoint)
                    child = Node(state=state, parent=node, action=action, priority=priority)
                    #priority = self.euclidean_distance(state, self.goalPoint)
                    frontier.put(child, priority)
                    max_frontier_size = max(max_frontier_size, len(frontier.elements))
                    space_complexity = max(space_complexity, max_frontier_size)


                    
# Number of goals you want to generate
num_goals = 5  
# Height of the grid
height = 15  
# Width of the grid
width = 15  

# Call the function with the required arguments
goals_list = generate_goals(num_goals, height, width)  # Generate goals as a list

# Create a copy of goals_list
goals_list_copy = goals_list.copy()

start = (0, 0)


import pygame
import time

# Initialize Pygame
pygame.init()

# Set up the display
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Loading...")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)

# Define fonts
font = pygame.font.SysFont(None, 48)

# Define loading text
loading_text = "LOADING..."

# Define loading progress variables
loading_progress = 0
loading_increment = 100 / len(loading_text)
loading_bar_width = 400
loading_bar_height = 30
loading_bar_rect = pygame.Rect((screen_width - loading_bar_width) // 2, screen_height // 2, 0, loading_bar_height)

# Display loading animation
for char in loading_text:
    loading_progress += loading_increment
    loading_bar_rect.width = loading_bar_width * loading_progress / 100

    # Draw loading bar
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLACK, (0, 0, screen_width, screen_height), 10)  # Black border around screen
    pygame.draw.rect(screen, BLACK, loading_bar_rect)
    
    # Draw loading text
    loading_text_surface = font.render(loading_text[:loading_text.index(char) + 1], True, BLACK)
    loading_text_rect = loading_text_surface.get_rect(center=(screen_width // 2, screen_height // 2 - 50))
    screen.blit(loading_text_surface, loading_text_rect)
    
    # Draw loading percentage
    percentage_text = f"{int(loading_progress)}%"
    percentage_surface = font.render(percentage_text, True, WHITE)
    percentage_rect = percentage_surface.get_rect(center=(screen_width // 2, screen_height // 2 + 50))
    screen.blit(percentage_surface, percentage_rect)
    
    # Update display
    pygame.display.flip()
    
    # Add a short delay
    time.sleep(0.5)

# Simulate loading process completion (just for demonstration)
time.sleep(1)

# Define transition variables
transition_duration = 1.0
transition_frames = int(transition_duration * 60)
fade_out_color = BLACK

# Fade out loading screen
for alpha in range(0, 255, 255 // transition_frames):
    screen.fill(WHITE)
    loading_text_surface.set_alpha(255 - alpha)
    screen.blit(loading_text_surface, loading_text_rect)
    pygame.display.flip()
    time.sleep(1 / 60)

# # Clean up loading resources
# pygame.quit()

# # Initialize Pygame again
# pygame.init()
# Set up the display
screen_width = 800
screen_height = 600

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Start and Goal States")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Define fonts
font = pygame.font.SysFont(None, 48)

# Define text
start_text = "START POINT:"
goal_text = "GOAL STATES:"
start_point = "(0, 0)"
# goals_list = ["(3, 4)", "(5, 6)", "(7, 8)"]  # Example goal states



# Define text animation variables
text_progress = 0
progress_increment = 100 / len(start_text)

# Define border
border_thickness = 5
border_rect = pygame.Rect(border_thickness // 2, border_thickness // 2, screen_width - border_thickness, screen_height - border_thickness)

# Display start point animation
for char in start_text:
    text_progress += progress_increment
    
    # Draw start point text
    screen.fill(WHITE)
    start_text_surface = font.render(start_text[:start_text.index(char) + 1], True, BLACK)
    start_text_rect = start_text_surface.get_rect(center=(screen_width // 2, screen_height // 4))
    screen.blit(start_text_surface, start_text_rect)
    pygame.draw.rect(screen, BLACK, border_rect, border_thickness)  # Draw border
    pygame.display.flip()
    
    # Add a short delay
    time.sleep(0.1)

# Add a short delay before displaying start point coordinates
time.sleep(1)

# Display start point coordinates
screen.fill(WHITE)
start_point_surface = font.render(f"{start_point}", True, BLACK)
start_point_rect = start_point_surface.get_rect(center=(screen_width // 2, screen_height // 4 + 50))
screen.blit(start_text_surface, start_text_rect)  # Re-draw start text
screen.blit(start_point_surface, start_point_rect)
pygame.draw.rect(screen, BLACK, border_rect, border_thickness)  # Draw border
pygame.display.flip()

# Add a short delay before displaying goal states animation
time.sleep(1)

# Display goal states animation
text_progress = 0
progress_increment = 100 / len(goal_text)
for char in goal_text:
    text_progress += progress_increment
    
    # Draw goal states text
    goal_text_surface = font.render(goal_text[:goal_text.index(char) + 1], True, BLACK)
    goal_text_rect = goal_text_surface.get_rect(center=(screen_width // 2, screen_height // 2))
    screen.fill(WHITE)
    screen.blit(start_text_surface, start_text_rect)  # Re-draw start text
    screen.blit(start_point_surface, start_point_rect)  # Re-draw start point
    screen.blit(goal_text_surface, goal_text_rect)
    pygame.draw.rect(screen, BLACK, border_rect, border_thickness)  # Draw border
    pygame.display.flip()
    
    # Add a short delay
    time.sleep(0.1)

# Display goal states
y_offset = 0
for i, goal in enumerate(goals_list, start=1):
    goal_surface = font.render(f"Delivery Point {i}: {goal}", True, BLACK)
    goal_rect = goal_surface.get_rect(center=(screen_width // 2, screen_height // 2 + y_offset + 50))
    screen.blit(goal_surface, goal_rect)
    y_offset += 50
pygame.draw.rect(screen, BLACK, border_rect, border_thickness)  # Draw border
pygame.display.flip()

# Wait for a short time
time.sleep(2)



# Define transition variables
transition_duration = 1.0
transition_frames = int(transition_duration * 60)
fade_out_color = BLACK

# Fade out start and goal display page
for alpha in range(0, 255, 255 // transition_frames):
    screen.fill(WHITE)
    start_text_surface.set_alpha(255 - alpha)
    screen.blit(start_text_surface, start_text_rect)
    goal_text_surface.set_alpha(255 - alpha)
    screen.blit(goal_text_surface, goal_text_rect)
    pygame.display.flip()
    time.sleep(1 / 60)


# maze = Maze(goals_list_copy, start, goals_list[0])  # Pass goals as a list, pop an element for maze goal




# # Initialize Pygame
# pygame.init()

# # Set up the display
# screen_width = 800
# screen_height = 600
# screen = pygame.display.set_mode((screen_width, screen_height))
# pygame.display.set_caption("Initial Maze Condition")

# # Define colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)

# # Define fonts
# font = pygame.font.SysFont(None, 48)

# # Define text
# text = "INITIAL MAZE CONDITION"

# # Define text animation variables
# text_progress = 0
# progress_increment = 100 / len(text)

# # Display text animation
# for i in range(len(text)):
#     # Draw text
#     screen.fill(WHITE)
#     text_surface = font.render(text[:i + 1], True, BLACK)
#     text_rect = text_surface.get_rect(center=(screen_width // 2, screen_height // 2))
#     screen.blit(text_surface, text_rect)
    
#     # Update display
#     pygame.display.update()  # Use update() instead of flip() for single updates
    
#     # Add a short delay
#     time.sleep(0.1)

# # Add a short delay before disappearing
# time.sleep(1)

# # Fade out text
# transition_duration = 1.0
# transition_frames = int(transition_duration * 60)
# for alpha in range(0, 255, 255 // transition_frames):
#     screen.fill(WHITE)
#     text_surface.set_alpha(255 - alpha)
#     screen.blit(text_surface, text_rect)
#     pygame.display.update()  # Update display
#     time.sleep(1 / 60)

    
# maze.print()
    

# # Initialize Pygame
# pygame.init()

# # Set up the display
# screen_width = 800
# screen_height = 600
# screen = pygame.display.set_mode((screen_width, screen_height))
# pygame.display.set_caption("Loading...")

# # Define colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)
# BLUE = (0, 0, 255)

# # Define fonts
# font = pygame.font.SysFont(None, 38)

# # Define text
# text = "SETTING OFF TOWARDS THE FIRST DELIVERY LOCATION!"
# text = text.upper()  # Convert text to uppercase

# # Define coordinate of the first delivery location
# # first_delivery_location = (1, 1)

# # Define loading progress variables
# loading_progress = 0
# loading_increment = 100 / len(text)
# loading_bar_width = 400
# loading_bar_height = 30
# loading_bar_rect = pygame.Rect((screen_width - loading_bar_width) // 2, screen_height // 2 + 50, 0, loading_bar_height)  # Adjusted position

# # Display loading animation
# for char in text:
#     loading_progress += loading_increment
#     loading_bar_rect.width = loading_bar_width * loading_progress / 100

#     # Draw loading bar
#     screen.fill(WHITE)
#     pygame.draw.rect(screen, BLACK, (0, 0, screen_width, screen_height), 10)  # Black border around screen
#     pygame.draw.rect(screen, BLACK, loading_bar_rect)
    
#     # Draw text
#     text_surface = font.render(text[:text.index(char) + 1], True, BLACK)
#     text_rect = text_surface.get_rect(center=(screen_width // 2, screen_height // 2 - 50))  # Adjusted position
#     screen.blit(text_surface, text_rect)
    
#     # Draw coordinate
#     coord_text = f"{goals_list[0]}"
#     coord_surface = font.render(coord_text, True, BLACK)
#     coord_rect = coord_surface.get_rect(center=(screen_width // 2, screen_height // 2))  # Adjusted position
#     coord_rect.y = text_rect.bottom + 10  # Position below the text
#     screen.blit(coord_surface, coord_rect)
    
#     # Draw loading percentage
#     percentage_text = f"{int(loading_progress)}%"
#     percentage_surface = font.render(percentage_text, True, WHITE)
#     percentage_rect = percentage_surface.get_rect(center=(screen_width // 2, screen_height // 2 + 100))
#     screen.blit(percentage_surface, percentage_rect)
    
#     # Update display
#     pygame.display.flip()
    
#     # Add a short delay
#     time.sleep(0.1)

# # Simulate loading process completion (just for demonstration)
# time.sleep(1)

# # Define transition variables
# transition_duration = 1.0
# transition_frames = int(transition_duration * 60)

# # Fade out loading screen
# for alpha in range(0, 255, 255 // transition_frames):
#     screen.fill(WHITE)
#     pygame.display.flip()
#     time.sleep(1 / 60)

# # # Clean up resources
# # pygame.quit()



# # print("Start Point: (0, 0)")
# # for i, goal in enumerate(goals_list, start=1):
# #     print(f"Delivery Point {i}: {goal}")
# # print("")



# # print("Solving...")
# maze.algo_a_star()

# maze.print()


# #print("Time complexity (States Explored):", maze.num_explored)
# # print("Solution:")




# # Initialize pygame
# pygame.init()

# # Set up the display
# WIDTH, HEIGHT = 800, 600
# screen = pygame.display.set_mode((WIDTH, HEIGHT))
# pygame.display.set_caption("Text Disappearing Animation")

# # Define colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)

# # Function to display text
# def display_text(text):
#     font = pygame.font.Font(None, 36)
#     text_surface = font.render(text, True, BLACK)
#     text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2))
#     screen.blit(text_surface, text_rect)

# # Function to animate text disappearance
# def animate_text_disappearance(text):
#     alpha = 255
#     while alpha > 0:
#         screen.fill(WHITE)
#         display_text(text)
#         # Reduce alpha gradually
#         alpha -= 5
#         if alpha < 0:
#             alpha = 0
#         overlay = pygame.Surface((WIDTH, HEIGHT))
#         overlay.set_alpha(alpha)
#         overlay.fill(WHITE)
#         screen.blit(overlay, (0, 0))
#         pygame.display.flip()
#         pygame.time.delay(50)  # Adjust the delay for animation speed
#     pygame.quit()

# # Get the text
# # maze_num_explored = 100  # Replace this with your actual value
# text = f"Time complexity (States Explored): {maze.num_explored}"

# # Display the text
# screen.fill(WHITE)
# display_text(text)
# pygame.display.flip()

# # Start the animation
# animate_text_disappearance(text)





# visited_goals = [maze.goalPoint]  
# goals_list.pop(0)



# # Initialize pygame
# pygame.init()

# # Set up the display
# WIDTH, HEIGHT = 800, 600
# screen = pygame.display.set_mode((WIDTH, HEIGHT))
# pygame.display.set_caption("Delivery Point Selection")

# # Define colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)
# RED = (255, 0, 0)

# # Function to display text
# def display_text(text, font_size, position, color=BLACK):
#     font = pygame.font.Font(None, font_size)
#     text_surface = font.render(text, True, color)
#     text_rect = text_surface.get_rect(center=position)
#     screen.blit(text_surface, text_rect)

# # Function to display delivery points
# def display_delivery_points(goals_list):
#     y_offset = 100
#     for i, goal in enumerate(goals_list, start=1):
#         display_text(f"{i}: {goal}", 36, (WIDTH//2, y_offset))
#         y_offset += 50

# # Function to display error message
# def display_error_message():
#     error_text = "Invalid choice. Please choose a valid delivery point."
#     display_text(error_text, 24, (WIDTH//2, HEIGHT-50), RED)
#     pygame.display.flip()
#     pygame.time.delay(1500)  # Display error message for 1.5 seconds

# # Function to get user choice
# def get_user_choice(goals_list):
#     while True:
#         for event in pygame.event.get():
#             if event.type == pygame.QUIT:
#                 pygame.quit()
#                 sys.exit()

#         screen.fill(WHITE)
#         display_text("Which delivery point would you prefer to go now:", 24, (WIDTH//2, 50))
#         display_delivery_points(goals_list)
#         pygame.display.flip()

#         choice = None
#         keys = pygame.key.get_pressed()
#         for i in range(1, min(5, len(goals_list)+1)):
#             if keys[getattr(pygame, f"K_{i}")]:
#                 choice = i
#                 break

#         if choice is not None:
#             return choice


visited_goals = [(0,0)]  

# Main loop
while goals_list:
    # Initialize pygame
    pygame.init()

    # Set up the display
    WIDTH, HEIGHT = 800, 600
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Delivery Point Selection")

    # Define colors
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)

    # Function to display text
    def display_text(text, font_size, position, color=BLACK):
        font = pygame.font.Font(None, font_size)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect(center=position)
        screen.blit(text_surface, text_rect)

    # Function to display delivery points
    def display_delivery_points(goals_list):
        y_offset = 100
        for i, goal in enumerate(goals_list, start=1):
            display_text(f"{i}: {goal}", 36, (WIDTH//2, y_offset))
            y_offset += 50

    # Function to display error message
    def display_error_message():
        error_text = "Invalid choice. Please choose a valid delivery point."
        display_text(error_text, 24, (WIDTH//2, HEIGHT-50), RED)
        pygame.display.flip()
        pygame.time.delay(1500)  # Display error message for 1.5 seconds

    # Function to get user choice
    def get_user_choice(goals_list):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

            screen.fill(WHITE)
            display_text("Which delivery point would you prefer to go now:", 24, (WIDTH//2, 50))
            display_delivery_points(goals_list)
            pygame.display.flip()

            choice = None
            keys = pygame.key.get_pressed()
           
            for i in range(1, min(6, len(goals_list)+1)):  # Change the upper limit to 6
                if keys[getattr(pygame, f"K_{i}")]:
                    choice = i
                    break

            if choice is not None:
                return choice
            
            
    choice = get_user_choice(goals_list)
    if 1 <= choice <= len(goals_list):
        start = visited_goals[-1]  
        goal_index = choice - 1  # Correct indexing
        new_goal = goals_list.pop(goal_index)  # Pop the selected goal from the list
        visited_goals.append(new_goal)  

        # Create a new Maze object with the updated start and goal
        maze = Maze(goals_list_copy, visited_goals[-2], new_goal)

        
        
        # Initialize Pygame
        pygame.init()

        # Set up the display
        screen_width = 800
        screen_height = 600
        screen = pygame.display.set_mode((screen_width, screen_height))
        pygame.display.set_caption("Initial Maze Condition")

        # Define colors
        WHITE = (255, 255, 255)
        BLACK = (0, 0, 0)

        # Define fonts
        font = pygame.font.SysFont(None, 48)

        # Define text
        text = "INITIAL MAZE CONDITION"

        # Define text animation variables
        text_progress = 0
        progress_increment = 100 / len(text)

        # Display text animation
        for i in range(len(text)):
            # Draw text
            screen.fill(WHITE)
            text_surface = font.render(text[:i + 1], True, BLACK)
            text_rect = text_surface.get_rect(center=(screen_width // 2, screen_height // 2))
            screen.blit(text_surface, text_rect)

            # Update display
            pygame.display.update()  # Use update() instead of flip() for single updates

            # Add a short delay
            time.sleep(0.1)

        # Add a short delay before disappearing
        time.sleep(1)

        # Fade out text
        transition_duration = 1.0
        transition_frames = int(transition_duration * 60)
        for alpha in range(0, 255, 255 // transition_frames):
            screen.fill(WHITE)
            text_surface.set_alpha(255 - alpha)
            screen.blit(text_surface, text_rect)
            pygame.display.update()  # Update display
            time.sleep(1 / 60)


    



        
#         # Initialize Pygame
#         pygame.init()

#         # Set up the display
#         screen_width = 800
#         screen_height = 600
#         screen = pygame.display.set_mode((screen_width, screen_height))
#         pygame.display.set_caption("Maze Condition Now")

#         # Define colors
#         WHITE = (255, 255, 255)
#         BLACK = (0, 0, 0)

#         # Define fonts
#         font = pygame.font.SysFont(None, 48)

#         # Define text
#         text = "MAZE CONDITION NOW"

#         # Define text animation variables
#         text_progress = 0
#         progress_increment = 100 / len(text)

#         # Display text animation
#         for char in text:
#             text_progress += progress_increment

#             # Draw text
#             screen.fill(WHITE)
#             text_surface = font.render(text[:text.index(char) + 1], True, BLACK)
#             text_rect = text_surface.get_rect(center=(screen_width // 2, screen_height // 2))
#             screen.blit(text_surface, text_rect)

#             # Update display
#             pygame.display.flip()

#             # Add a short delay
#             time.sleep(0.1)

#         # Add a short delay before disappearing
#         time.sleep(1)

#         # Fade out text
#         transition_duration = 1.0
#         transition_frames = int(transition_duration * 60)
#         for alpha in range(0, 255, 255 // transition_frames):
#             screen.fill(WHITE)
#             text_surface.set_alpha(255 - alpha)
#             screen.blit(text_surface, text_rect)
#             pygame.display.flip()
#             time.sleep(1 / 60)







        
#         print("Maze:")
        maze.print()
        
        
        
        # Initialize Pygame
        pygame.init()

        # Set up the display
        screen_width = 800
        screen_height = 600
        screen = pygame.display.set_mode((screen_width, screen_height))
        pygame.display.set_caption("Loading...")

        # Define colors
        WHITE = (255, 255, 255)
        BLACK = (0, 0, 0)
        BLUE = (0, 0, 255)

        # Define fonts
        font = pygame.font.SysFont(None, 38)

        # Define text
        text = f"SETTING OFF TOWARDS THE DELIVERY LOCATION!"
        text = text.upper()  # Convert text to uppercase

        # Define coordinate of the first delivery location
        # first_delivery_location = (1, 1)

        # Define loading progress variables
        loading_progress = 0
        loading_increment = 100 / len(text)
        loading_bar_width = 400
        loading_bar_height = 30
        loading_bar_rect = pygame.Rect((screen_width - loading_bar_width) // 2, screen_height // 2 + 50, 0, loading_bar_height)  # Adjusted position

        # Display loading animation
        for char in text:
            loading_progress += loading_increment
            loading_bar_rect.width = loading_bar_width * loading_progress / 100

            # Draw loading bar
            screen.fill(WHITE)
            pygame.draw.rect(screen, BLACK, (0, 0, screen_width, screen_height), 10)  # Black border around screen
            pygame.draw.rect(screen, BLACK, loading_bar_rect)

            # Draw text
            text_surface = font.render(text[:text.index(char) + 1], True, BLACK)
            text_rect = text_surface.get_rect(center=(screen_width // 2, screen_height // 2 - 50))  # Adjusted position
            screen.blit(text_surface, text_rect)

            # Draw coordinate
            coord_text = f"{new_goal}"
            coord_surface = font.render(coord_text, True, BLACK)
            coord_rect = coord_surface.get_rect(center=(screen_width // 2, screen_height // 2))  # Adjusted position
            coord_rect.y = text_rect.bottom + 10  # Position below the text
            screen.blit(coord_surface, coord_rect)

            # Draw loading percentage
            percentage_text = f"{int(loading_progress)}%"
            percentage_surface = font.render(percentage_text, True, WHITE)
            percentage_rect = percentage_surface.get_rect(center=(screen_width // 2, screen_height // 2 + 100))
            screen.blit(percentage_surface, percentage_rect)

            # Update display
            pygame.display.flip()

            # Add a short delay
            time.sleep(0.1)

        # Simulate loading process completion (just for demonstration)
        time.sleep(1)

        # Define transition variables
        transition_duration = 1.0
        transition_frames = int(transition_duration * 60)

        # Fade out loading screen
        for alpha in range(0, 255, 255 // transition_frames):
            screen.fill(WHITE)
            pygame.display.flip()
            time.sleep(1 / 60)

        # # Clean up resources
        # pygame.quit()

        
        
        
#         print("Solving...")
        maze.greedy_bfs()
        
        
        
        
        
        
        
        
#         print("Time complexity (States Explored):", maze.num_explored)
#         print("Solution:")
        maze.print()
    
    
        # Initialize pygame
        pygame.init()

        # Set up the display
        WIDTH, HEIGHT = 800, 600
        screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Text Disappearing Animation")

        # Define colors
        WHITE = (255, 255, 255)
        BLACK = (0, 0, 0)

        # Function to display text
        def display_text(text):
            font = pygame.font.Font(None, 36)
            text_surface = font.render(text, True, BLACK)
            text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2))
            screen.blit(text_surface, text_rect)

        # Function to animate text disappearance
        def animate_text_disappearance(text):
            alpha = 255
            while alpha > 0:
                screen.fill(WHITE)
                display_text(text)
                # Reduce alpha gradually
                alpha -= 5
                if alpha < 0:
                    alpha = 0
                overlay = pygame.Surface((WIDTH, HEIGHT))
                overlay.set_alpha(alpha)
                overlay.fill(WHITE)
                screen.blit(overlay, (0, 0))
                pygame.display.flip()
                pygame.time.delay(50)  # Adjust the delay for animation speed
            pygame.quit()

        # Get the text
        # maze_num_explored = 100  # Replace this with your actual value
        text = f"Time complexity (States Explored): {maze.num_explored}"

        # Display the text
        screen.fill(WHITE)
        display_text(text)
        pygame.display.flip()

        # Start the animation
        animate_text_disappearance(text)

        
    
    else:
        display_error_message()
    

# print("All delivery points have been visited.")
# Function to display text
def display_text(text, font_size, color, x, y):
    font = pygame.font.Font(None, font_size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect(center=(x, y))
    screen.blit(text_surface, text_rect)

# Initialize Pygame
pygame.init()

# Set up the display
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("End of Delivery")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Define fonts
font_size = 48
font_color = BLACK

# Display "All delivery points have been visited."
screen.fill(WHITE)
display_text("All delivery points have been visited.", font_size, font_color, screen_width // 2, screen_height // 2)
pygame.display.flip()
time.sleep(2)  # Display for 2 seconds

# Display "GOOD BYE"
screen.fill(WHITE)
display_text("GOOD BYE", font_size, font_color, screen_width // 2, screen_height // 2)
pygame.display.flip()
time.sleep(2)  # Display for 2 seconds

# Display "PHIR MILAIN NA MILAIN" with animation
text = "PHIR MILAIN NA MILAIN"
for i in range(len(text) + 1):
    screen.fill(WHITE)
    display_text(text[:i], font_size, font_color, screen_width // 2, screen_height // 2)
    pygame.display.flip()
    time.sleep(0.1)  # Adjust the delay for animation speed

# Add a short delay before disappearing
time.sleep(1)

# Fade out text
transition_duration = 1.0
transition_frames = int(transition_duration * 60)
for alpha in range(0, 255, 255 // transition_frames):
    screen.fill(WHITE)
    display_text(text, font_size, font_color, screen_width // 2, screen_height // 2)
    pygame.display.flip()
    time.sleep(1 / 60)

# Clean up resources
pygame.quit()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
